In [1]:
import numpy as np
from floquet_simulations.plot_functions import PlotParams

from floquet_simulations.hamiltonians import GetEvalsAndEvecsGen

from scipy.integrate import solve_ivp
from floquet_simulations.hamiltonians import RoundComplex

from floquet_simulations.plot_functions import PlotAbsRealImagHamiltonian
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.special import jv
from pathlib import Path
import pandas as pd
import math



In [2]:
def square_2D_edges(rows, columns):
    H0 = np.zeros((rows*columns,rows*columns))
    # 
    links = []
    for y in range(rows):
        for x in range(columns-1):
            links.append((x + y*columns, x+1 + y* columns))
    for y in range(rows-1):
        for x in range(columns):
            links.append((x + columns*y, 
                          x+columns + columns*y))
    return(links)

def H0_2D(rows, columns):
    H0 = np.zeros((rows*columns,rows*columns))
    links = square_2D_edges(rows, columns)
    for link in links:
        H0[link[0], link[1]] = -1
        H0[link[1], link[0]] = -1
    return H0

def HT(H0, funcs, sites, t):

    for func, site in zip(funcs, sites):
        H0[site-1,site-1] = func(t)
    return H0

def FT(t, psi, H0, funcs, sites):
    H = HT(H0, funcs, sites, t)
    return -1j*np.dot(H, psi)

def SolveSchrodinger(H0, funcs, sites, tspan, n_timesteps, psi0):
    
    rtol=1e-11
    # points to calculate the matter wave at
    t_eval = np.linspace(tspan[0], tspan[1], n_timesteps+1, endpoint=True)
    sol = solve_ivp(lambda t,psi: FT(t, psi, H0, funcs, sites), 
            t_span=tspan, y0=psi0, rtol=rtol, 
            atol=rtol, t_eval=t_eval,
            method='RK45')
    sol=sol.y
    return sol


def CreateHF(H0, funcs, sites, T, num_sites, t0=0, hermitian_accuracy_dp=7): 
    """
    t0 is fraction of T
    """

    #sanity check
    assert(len(H0) == num_sites)
    
    tspan = (t0*T,T+t0*T)
    UT = np.zeros([num_sites, num_sites], dtype=np.complex_)
    n_timesteps = 100
    
    for A_site_start in range(num_sites):
        psi0 = np.zeros(num_sites, dtype=np.complex_); psi0[A_site_start] = 1
        sol = SolveSchrodinger(H0, funcs, sites, tspan, n_timesteps, psi0)
        UT[:,A_site_start]=sol[:,-1] 
    
    # evals_U, evecs = eig(UT)
    evals_U, evecs = GetEvalsAndEvecsGen(UT) #evals can be imaginary
    evals_H = 1j / T *np.log(evals_U)
    
    HF = np.zeros([num_sites,num_sites], dtype=np.complex_)
    for i in range(num_sites):
        term = evals_H[i]*np.outer(evecs[:,i], np.conj(evecs[:,i]))
        HF = HF+term

    HF = RoundComplex(HF, hermitian_accuracy_dp)
    # assert(np.all(0 == (HFr - np.conj(HFr.T))))
    return UT, HF
    # if np.all(0 == (HF - np.conj(HF.T))):
    #     return UT, HF
    # else:
    #     return np.nan, np.nan


def get_A_and_xi_ij(Ai, Aj, phii, phij):
# note we expect  Aj to be negative!!!!!
    if (np.round(Ai,7) ==np.round(Aj,7)) and ((np.round(phii/pi,7) == np.round(phij/pi,7)) or (np.round(phii/pi,7)== np.round( phij/pi + 2,7)) or (np.round(phii/pi + 2,7)== np.round( phij/pi,7))):
        return 0,0
    else:
        Aij =  np.sqrt(Aj**2 + Ai**2 - 2*Ai*Aj*np.cos(phii - phij))

        cos_result = (-Aj * np.cos(phij) + Ai*np.cos(phii))/(Aij)
        sin_result = (-Aj*np.sin(phij) + Ai*np.sin(phii))/(Aij)
        cos_args = np.round(np.array( [np.arccos(cos_result)/np.pi, 2 - np.arccos(cos_result)/np.pi])  , 5)
        sin_args = np.round(np.array([i + 2 if i<0 else i for i in [np.arcsin(sin_result)/np.pi, 1 - np.arcsin(sin_result)/np.pi] ]), 5)

        return Aij, np.intersect1d(cos_args, sin_args)[0]*np.pi
    

def Vt(m,n, kappa, omega, t, Delta, delta):
    phi0 = 0
    fr = np.cos(m*np.pi/2 - np.pi/4)
    fb = np.cos(m*np.pi/2 + np.pi/4)
    gr = phi0 - n*np.pi/2
    gb = n*np.pi/2 - phi0 - np.pi/2
    return kappa*(fr*np.cos(omega*t + gr) + fb*np.cos(omega*t + gb)) + Delta/2*(-1)**m + delta/2*((-1)**m + (-1)**n)


L_x = 4
plaquette_sides =  [(0,0), (L_x,1), (L_x - 1, L_x + 1), (-1,L_x)]
plaquette1 = [(1+i, 0+j) for (i,j) in plaquette_sides]
plaquette2 = [(2+i, 1+j) for (i,j) in plaquette_sides]
plaquette3 = [(3+i, 2+j) for (i,j) in plaquette_sides]

plaquette4 = [(5+i, 4+j) for (i,j) in plaquette_sides]
plaquette5 = [(6+i, 5+j) for (i,j) in plaquette_sides]
plaquette6 = [(7+i, 6+j) for (i,j) in plaquette_sides]

plaquette7 = [(9+i, 8+j) for (i,j) in plaquette_sides]
plaquette8= [(10+i, 9+j) for (i,j) in plaquette_sides]
plaquette9 = [(11+i, 10+j) for (i,j) in plaquette_sides]

## get As from travelling waves

In [ ]:

    
    # omega = 10
omega = 50
Delta = omega
delta = 0#omega*0.001
kappa = 10

L_x = 4
L_y = 2
N = L_x*L_y

T = 2*np.pi/omega
m_site = [0,1,2,3,0,1,2,3]
n_site = [0,0,0,0,1,1,1,1]

site_drives_and_offsets = np.array([[get_A_and_xi_ij(kappa*np.cos(m_it*np.pi/2 - np.pi/4),-kappa*np.cos(m_it*np.pi/2 + np.pi/4), - n_it*np.pi/2, n_it*np.pi/2  - np.pi/2) for m_it in range(L_x)] for n_it in range(L_y)])

merged_As = site_drives_and_offsets[:,:,0]
merged_varphis = site_drives_and_offsets[:,:,1]


nus = np.array([0,1,0,1,0,1,0,1
                ,0,1,0,1,0,1,0,1
                ])*omega



# merged_varphis = np.array(list(zip(*site_drives_and_offsets))[1])
# merged_As =  np.array(list(zip(*site_drives_and_offsets))[0])

# single phi3 working version

In [8]:
# omega = 10
# A1 = 40; A2 = 10; A3 = 20; A4 = 40
omega = 50
# delta = omega
# phi1 =0.4*np.pi
# phi2  = 0.7*np.pi# 0.75*2*np.pi
# phi3 = 0.9*np.pi#0.825*2*np.pi
L_x = 4
L_y = 2
N = L_x*L_y
T = 2*np.pi/omega
sites = np.arange(1,L_x*L_y + 1,1)

# As = [A1, A2, A1, A2, A3, A4, A3, A4
#       , A1, A3, A1, A3, A2, A4, A2, A4
#       ]

nus = np.array([0,1,0,1,0,1,0,1
                ,0,1,0,1,0,1,0,1
                ])*omega


# varphis = [0, 0, 0, 0, 0, phi1, phi1+phi2,phi3,
#             #    ,0, 0, 0, 0,0, phi1, phi1+phi2, phi1+phi2+phi3
#             ]

Delta = omega

m_site = [0,1,2,3,0,1,2,3]
n_site = [0,0,0,0,1,1,1,1]
As = [10,10,10,10,10,10,10,10]
varphis = np.array([1.75, 0.25, 0.75, 1.25, 1.75, 1.25, 0.75, 0.25])
# nus = np.array([
#     Delta/2*(-1)**m_site[0] ,
#        Delta/2*(-1)**m_site[1],
#        Delta/2*(-1)**m_site[2] ,
#        Delta/2*(-1)**m_site[3],
#        Delta/2*(-1)**m_site[4],
#        Delta/2*(-1)**m_site[5] ,
#        Delta/2*(-1)**m_site[6] ,
#        Delta/2*(-1)**m_site[7] ])


funcs = [lambda x: As[0]*np.cos(omega*x + varphis[0]) + nus[0],
            lambda x: As[1]*np.cos(omega*x + varphis[1]) + nus[1],
            lambda x: As[2]*np.cos(omega*x + varphis[2]) + nus[2],
            lambda x: As[3]*np.cos(omega*x + varphis[3]) + nus[3],
            lambda x: As[4]*np.cos(omega*x + varphis[4]) + nus[4],
            lambda x: As[5]*np.cos(omega*x + varphis[5]) + nus[5],
            lambda x: As[6]*np.cos(omega*x + varphis[6]) + nus[6],
            lambda x: As[7]*np.cos(omega*x + varphis[7]) + nus[7],
            lambda x: As[8]*np.cos(omega*x + varphis[8]) + nus[8]
            ,lambda x: As[9]*np.cos(omega*x + varphis[9]) + nus[9],
            lambda x: As[10]*np.cos(omega*x + varphis[10]) + nus[10],
            lambda x: As[11]*np.cos(omega*x + varphis[11]) + nus[11],
            lambda x: As[12]*np.cos(omega*x + varphis[12]) + nus[12],
            lambda x: As[13]*np.cos(omega*x + varphis[13]) + nus[13],
            lambda x: As[14]*np.cos(omega*x + varphis[14]) + nus[14],
            lambda x: As[15]*np.cos(omega*x + varphis[15]) + nus[15]
            ]


_, HF = CreateHF(H0_2D(L_y,L_x), funcs, sites, T, num_sites=L_x*L_y, t0=0, hermitian_accuracy_dp=7)

# for links 1-10
Bs = np.zeros((N, N))
xis = np.zeros((N, N))
HF_theory = np.zeros((N, N), dtype=np.complex128)
#do horizontal links
for ny in range(L_y):
    for nx in range(L_x-1):
        
        jj = nx + L_x*ny
        ii = nx+L_x*ny + 1

        nu_ij = (nus[ii] - nus[jj])/omega
        
        B_ij, xi_ij= get_A_and_xi_ij(As[ii], As[jj], varphis[ii], varphis[jj])
        t_ij = (float(-1))**nu_ij*jv(nu_ij, B_ij/omega)*np.exp((-1)*nu_ij*1j*xi_ij)

        Bs[ii, jj] = B_ij
        xis[ii, jj] = xi_ij

        HF_theory[ii,jj] = t_ij
        HF_theory[jj, ii] = np.conj(t_ij)

#do vertical links
for ny in range(L_y-1):
    for nx in range(L_x):
        jj = nx + L_x*ny
        ii = nx+L_x*ny + L_x

        B_ij, xi_ij= get_A_and_xi_ij(As[ii], As[jj],  varphis[ii], varphis[jj])
        t_ij = jv(0, B_ij/omega)

        Bs[ii,jj] = B_ij
        xis[ii, jj] = xi_ij
        HF_theory[ii,jj] = t_ij
        HF_theory[jj, ii] = np.conj(t_ij)

tunnelling_indices = [(1,2), (2,3), (3,4), (1,5), (2,6), (3,7), (4,8), (5,6), (6,7), (7,8)]
exp_list = [-HF[i-1, j-1] for (i,j) in tunnelling_indices]
theory_list =  [HF_theory[i-1, j-1] for (i,j) in tunnelling_indices]
tunnellings = pd.DataFrame({"index": tunnelling_indices,
                            "theory":np.abs(np.array(theory_list)),
                            "exp":np.abs(np.array(exp_list))
})

fluxes = pd.DataFrame({"theory":[np.angle(math.prod([HF_theory[i, j] for (i,j) in plaquette1]))/np.pi,
                                np.angle(math.prod([HF_theory[i, j] for (i,j) in plaquette2]))/np.pi,
                                np.angle(math.prod([HF_theory[i, j] for (i,j) in plaquette3]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette4]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette5]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette6]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette7]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette8]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette9]))/np.pi
                                ],

                    "exp":   [np.angle(math.prod([HF[i, j] for (i,j) in plaquette1]))/np.pi,
                                np.angle(math.prod([HF[i, j] for (i,j) in plaquette2]))/np.pi,
                                np.angle(math.prod([HF[i, j] for (i,j) in plaquette3]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette4]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette5]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette6]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette7]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette8]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette9]))/np.pi
                                ]
                    })

with pd.option_context('display.max_rows', None,
                    'display.max_columns', 7,
                    'display.precision', 5,
                    ):
    print(fluxes)
    print(tunnellings)

    theory      exp
0  0.15915  0.16105
1  0.84084  0.81890
2  0.84084  0.82663
    index   theory      exp
0  (1, 2)  0.13506  0.13369
1  (2, 3)  0.04942  0.04838
2  (3, 4)  0.04942  0.05078
3  (1, 5)  1.00000  0.99999
4  (2, 6)  0.99083  0.99082
5  (3, 7)  1.00000  0.99998
6  (4, 8)  0.99083  0.99081
7  (5, 6)  0.04942  0.04769
8  (6, 7)  0.04942  0.05145
9  (7, 8)  0.04942  0.04871


# iterated phi3 working version

In [ ]:
# omega = 10
A1 = 40; A2 = 7; A3 = 7; A4 = 40
omega = 100
delta = omega
N_phis = 50
phi1s = np.linspace(0,2*np.pi, N_phis)
phi2  = 0*np.pi# 0.75*2*np.pi
phi3 = 0*np.pi#0.825*2*np.pi
L_x = 4
L_y = 2
N = L_x*L_y
T = 2*np.pi/omega
sites = np.arange(1,L_x*L_y + 1,1)
HFs = np.zeros((N,N,N_phis), dtype = np.complex128)
HFs_theory = np.zeros((N,N,N_phis), dtype=np.complex128)


As = [A1, A2, A1, A2, A3, A4, A3, A4
      , A1, A3, A1, A3, A2, A4, A2, A4
      ]
nus = np.array([0,1,0,1,0,1,0,1
                ,0,1,0,1,0,1,0,1
                ])*omega

for nphi, phi1 in enumerate(phi1s):
    varphis = [0, 0, 0, 0, 0, phi1, 0,0,
            #    ,0, 0, 0, 0,0, phi1, phi1+phi2, phi1+phi2+phi3
            ]



    funcs = [lambda x: As[0]*np.cos(omega*x + varphis[0]) + nus[0],
                lambda x: As[1]*np.cos(omega*x + varphis[1]) + nus[1],
                lambda x: As[2]*np.cos(omega*x + varphis[2]) + nus[2],
                lambda x: As[3]*np.cos(omega*x + varphis[3]) + nus[3],
                lambda x: As[4]*np.cos(omega*x + varphis[4]) + nus[4],
                lambda x: As[5]*np.cos(omega*x + varphis[5]) + nus[5],
                lambda x: As[6]*np.cos(omega*x + varphis[6]) + nus[6],
                lambda x: As[7]*np.cos(omega*x + varphis[7]) + nus[7],
                lambda x: As[8]*np.cos(omega*x + varphis[8]) + nus[8]
                ,lambda x: As[9]*np.cos(omega*x + varphis[9]) + nus[9],
                lambda x: As[10]*np.cos(omega*x + varphis[10]) + nus[10],
                lambda x: As[11]*np.cos(omega*x + varphis[11]) + nus[11],
                lambda x: As[12]*np.cos(omega*x + varphis[12]) + nus[12],
                lambda x: As[13]*np.cos(omega*x + varphis[13]) + nus[13],
                lambda x: As[14]*np.cos(omega*x + varphis[14]) + nus[14],
                lambda x: As[15]*np.cos(omega*x + varphis[15]) + nus[15]
                ]


    _, HF = CreateHF(H0_2D(L_y,L_x), funcs, sites, T, num_sites=L_x*L_y, t0=0, hermitian_accuracy_dp=7)
    HFs[:,:,nphi] = HF

    # for links 1-10
    Bs = np.zeros((N, N))
    xis = np.zeros((N, N))
    HF_theory = np.zeros((N, N), dtype=np.complex128)
    #do horizontal links
    for ny in range(L_y):
        for nx in range(L_x-1):
            
            jj = nx + L_x*ny
            ii = nx+L_x*ny + 1

            nu_ij = (nus[ii] - nus[jj])/omega
            
            B_ij, xi_ij= get_A_and_xi_ij(As[ii], As[jj], varphis[ii], varphis[jj])
            t_ij = (float(-1))**nu_ij*jv(nu_ij, B_ij/omega)*np.exp((-1)*nu_ij*1j*xi_ij)

            Bs[ii, jj] = B_ij
            xis[ii, jj] = xi_ij

            HF_theory[ii,jj] = t_ij
            HF_theory[jj, ii] = np.conj(t_ij)

    #do vertical links
    for ny in range(L_y-1):
        for nx in range(L_x):
            jj = nx + L_x*ny
            ii = nx+L_x*ny + L_x

            B_ij, xi_ij= get_A_and_xi_ij(As[ii], As[jj],  varphis[ii], varphis[jj])
            t_ij = jv(0, B_ij/omega)

            Bs[ii,jj] = B_ij
            xis[ii, jj] = xi_ij
            HF_theory[ii,jj] = t_ij
            HF_theory[jj, ii] = np.conj(t_ij)

    HFs_theory[:,:,nphi] = HF_theory
    



In [ ]:
cm_unit = 1/2.54
figsize = (10,10)
markersize1 = 9
markersize2 = 3
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize = (figsize[0]*cm_unit*3, figsize[1]*cm_unit))
ax[0].plot(phi1s,  np.angle(math.prod([HFs[i, j, :] for (i,j) in plaquette1]))/np.pi, '.',label = r"$\Phi_1^e$", ms = markersize1)
ax[1].plot(phi1s,  np.angle(math.prod([HFs[i, j, :] for (i,j) in plaquette2]))/np.pi, '.',label = r"$\Phi_2^e$", ms = markersize1)
ax[2].plot(phi1s,  np.angle(math.prod([HFs[i, j, :] for (i,j) in plaquette3]))/np.pi, '.',label = r"$\Phi_3^e$", ms = markersize1)
ax[0].plot(phi1s,  np.angle(math.prod([HFs_theory[i, j, :] for (i,j) in plaquette1]))/np.pi, '.',label = r"$\Phi_1^t$", ms = markersize2)
ax[1].plot(phi1s,  np.angle(math.prod([HFs_theory[i, j, :] for (i,j) in plaquette2]))/np.pi, '.',label = r"$\Phi_2^t$", ms = markersize2)
ax[2].plot(phi1s,  np.angle(math.prod([HFs_theory[i, j, :] for (i,j) in plaquette3]))/np.pi, '.',label = r"$\Phi_3^t$", ms = markersize2)
ax[0].set_ylabel(r"$\Phi_i / \pi$", rotation = 0)
for i in range(3):
    ax[i].legend()
    ax[i].set_xlabel(r"$\varphi_1$")
plt.show()


fig, ax = plt.subplots(nrows = 1, ncols = 4, figsize = (figsize[0]*cm_unit*4, figsize[1]*cm_unit))
ax[0].plot(phi1s, np.abs(np.array(HFs[1,0,:])), '.', label = r"$|J_{21}|^{me}$", ms = markersize1)
ax[1].plot(phi1s, np.abs(np.array(HFs[5,1,:])), '.', label = r"$|J_{62}|^{me}$", ms = markersize1)
ax[2].plot(phi1s, np.abs(np.array(HFs[4,5,:])), '.', label = r"$|J_{56}|^{me}$", ms = markersize1)
ax[3].plot(phi1s, np.abs(np.array(HFs[0,4,:])), '.', label = r"$|J_{15}|^{me}$", ms = markersize1)
ax[0].plot(phi1s, np.abs(np.array(HFs_theory[1,0,:])), '.', label = r"$|J_{21}|^{mt}$", ms = markersize2)
ax[1].plot(phi1s, np.abs(np.array(HFs_theory[5,1,:])), '.', label = r"$|J_{62}|^{mt}$", ms = markersize2)
ax[2].plot(phi1s, np.abs(np.array(HFs_theory[4,5,:])), '.', label = r"$|J_{56}|^{mt}$", ms = markersize2)
ax[3].plot(phi1s, np.abs(np.array(HFs_theory[0,4,:])), '.', label = r"$|J_{15}|^{mt}$", ms = markersize2)
ax[0].set_ylabel(r"$|J_{ij}|$", rotation=0)
for i in range(4):
    ax[i].legend()
    ax[i].set_xlabel(r"$\varphi_1$")
plt.show()

In [ ]:
# omega = 10
A1 = 40; A2 = 70; A3 = 50; A4 = 40
omega = 50
delta = omega
phi1 = 0.4*np.pi
phi2  = 0.4*np.pi# 0.75*2*np.pi
phi3 = 0.9*np.pi#0.825*2*np.pi

L_x = 4
L_y = 4
N = L_x*L_y

T = 2*np.pi/omega
sites = np.arange(1,L_x*L_y + 1,1)
As = [A1, A2, A1, A2, A3, A4, A3, A4, A1, A2, A1, A2, A3, A4, A3, A4]
nus = np.array([0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1])*omega

varphis = [0, 0, 0, 0, 0, phi1, phi1+phi2, phi1+phi2+phi3,0, 0, 0, 0,0, phi1, phi1+phi2, phi1+phi2+phi3]

funcs = [lambda x: As[0]*np.cos(omega*x + varphis[0]) + nus[0],
            lambda x: As[1]*np.cos(omega*x + varphis[1]) + nus[1],
            lambda x: As[2]*np.cos(omega*x + varphis[2]) + nus[2],
            lambda x: As[3]*np.cos(omega*x + varphis[3]) + nus[3],
            lambda x: As[4]*np.cos(omega*x + varphis[4]) + nus[4],
            lambda x: As[5]*np.cos(omega*x + varphis[5]) + nus[5],
            lambda x: As[6]*np.cos(omega*x + varphis[6]) + nus[6],
            lambda x: As[7]*np.cos(omega*x + varphis[7]) + nus[7],
            lambda x: As[8]*np.cos(omega*x + varphis[8]) + nus[8],
            lambda x: As[9]*np.cos(omega*x + varphis[9]) + nus[9],
            lambda x: As[10]*np.cos(omega*x + varphis[10]) + nus[10],
            lambda x: As[11]*np.cos(omega*x + varphis[11]) + nus[11],
            lambda x: As[12]*np.cos(omega*x + varphis[12]) + nus[12],
            lambda x: As[13]*np.cos(omega*x + varphis[13]) + nus[13],
            lambda x: As[14]*np.cos(omega*x + varphis[14]) + nus[14],
            lambda x: As[15]*np.cos(omega*x + varphis[15]) + nus[15]]


_, HF = CreateHF(H0_2D(L_y,L_x), funcs, sites, T, num_sites=L_x*L_y, t0=0, hermitian_accuracy_dp=7)

# for links 1-10
Bs = np.zeros((N, N))
xis = np.zeros((N, N))
tunnellings = np.zeros((N, N), dtype=np.complex128)
#do horizontal links
for ny in range(L_y):
    for nx in range(L_x-1):
        
        jj = nx + L_x*ny
        ii = nx+L_x*ny + 1

        nu_ij = (nus[ii] - nus[jj])/omega
        
        B_ij, xi_ij= get_A_and_xi_ij(As[ii], As[jj], omega, varphis[ii], varphis[jj])
        t_ij = (float(-1))**nu_ij*jv(nu_ij, B_ij)*np.exp((-1)*nu_ij*1j*xi_ij)

        Bs[ii, jj] = B_ij
        xis[ii, jj] = xi_ij

        tunnellings[ii,jj] = t_ij
        tunnellings[jj, ii] = np.conj(t_ij)

#do vertical links
for ny in range(L_y-1):
    for nx in range(L_x):
        jj = nx + L_x*ny
        ii = nx+L_x*ny + L_x

        B_ij, xi_ij= get_A_and_xi_ij(As[ii], As[jj], omega, varphis[ii], varphis[jj])
        t_ij = jv(0, B_ij)

        Bs[ii,jj] = B_ij
        xis[ii, jj] = xi_ij
        tunnellings[ii,jj] = t_ij
        tunnellings[jj, ii] = np.conj(t_ij)
# 
plaquette_sides =  [(0,0), (L_x,1), (L_x - 1, L_x + 1), (-1,L_x)]
plaquette1 = [(1+i, 0+j) for (i,j) in plaquette_sides]
plaquette2 = [(2+i, 1+j) for (i,j) in plaquette_sides]
plaquette3 = [(3+i, 2+j) for (i,j) in plaquette_sides]

plaquette4 = [(5+i, 4+j) for (i,j) in plaquette_sides]
plaquette5 = [(6+i, 5+j) for (i,j) in plaquette_sides]
plaquette6 = [(7+i, 6+j) for (i,j) in plaquette_sides]

plaquette7 = [(9+i, 8+j) for (i,j) in plaquette_sides]
plaquette8= [(10+i, 9+j) for (i,j) in plaquette_sides]
plaquette9 = [(11+i, 10+j) for (i,j) in plaquette_sides]


fluxes = pd.DataFrame({"theory":[np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette1]))/np.pi,
                                 np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette2]))/np.pi,
                                 np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette3]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette4]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette5]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette6]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette7]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette8]))/np.pi,
                                #  np.angle(math.prod([tunnellings[i, j] for (i,j) in plaquette9]))/np.pi
                                 ],

                       "exp":   [np.angle(math.prod([HF[i, j] for (i,j) in plaquette1]))/np.pi,
                                 np.angle(math.prod([HF[i, j] for (i,j) in plaquette2]))/np.pi,
                                 np.angle(math.prod([HF[i, j] for (i,j) in plaquette3]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette4]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette5]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette6]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette7]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette8]))/np.pi,
                                #  np.angle(math.prod([HF[i, j] for (i,j) in plaquette9]))/np.pi
                                 ]
                       })

with pd.option_context('display.max_rows', None,
                       'display.max_columns', 7,
                       'display.precision', 5,
                       ):
    print(fluxes)



In [ ]:
A1 = 4
As = [A1, A2, A1, A2, A3, A4, A3, A4, A1, A2, A1, A2, A3, A4, A3, A4]
funcs = [lambda x: As[0]*np.cos(omega*x + varphis[0]) + nus[0]]

funcs2 = []
for i_site in range(1):
    print
    funcs2.append(lambda t: As[i_site]*np.cos(omega*t + varphis[i_site]) + nus[i_site])

t = np.linspace(0,2*np.pi,100)
fig, ax = plt.subplots()
ax.plot(t, funcs[0](t), label="func")
ax.plot(t, funcs2[0](t), label="func2")
plt.legend()
plt.show()

In [ ]:
figsize = (25, 8)
colourbar_pad=0.4
colourbar_size_percentage=5
norm = mpl.colors.Normalize(vmin=-1, vmax=1)
cmap = LinearSegmentedColormap.from_list('custom hamiltonians', ['#006F63', "#FFFFFF", '#F78320'], N=256)
apply = [
            np.abs, 
            np.real, np.imag]
labels = [
            r'$\mathrm{Abs}\{G_{n,m}\}$', 
            r'$\mathrm{Re}\{G_{n,m}\}$',
            r'$\mathrm{Imag}\{G_{n,m}\}$'
            ]
cm_unit = 1/2.54
fig, ax = plt.subplots(nrows=1, ncols=len(apply), sharey=True, constrained_layout=True, 
                        figsize=(figsize[0]*cm_unit, figsize[1]*cm_unit))
fig.suptitle(rf"$\omega={omega}, A_1 ={A1}, A_2 = {A2}, A_3 = {A3}, A_4 = {A4}$")
for n1, f in enumerate(apply):
    pcm = ax[n1].matshow(f(HF), interpolation='none', cmap=cmap,  norm=norm)
    ax[n1].set_title(labels[n1])
    ax[n1].tick_params(axis="x", bottom=True, top=False, labelbottom=True, 
        labeltop=False)  
    ax[n1].set_xlabel('m')
ax[0].set_ylabel('n', rotation=0, labelpad=10)
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size=f"{colourbar_size_percentage}%", pad=colourbar_pad)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)# label="unweighted graph distance")
plt.show()

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=len(apply), sharey=True, constrained_layout=True, 
                        figsize=(figsize[0]*cm_unit, figsize[1]*cm_unit))
fig.suptitle(rf"$\omega={omega}, A_1 ={A1}, A_2 = {A2}, A_3 = {A3}, A_4 = {A4}$")
for n1, f in enumerate(apply):
    pcm = ax[n1].matshow(f(tunnellings), interpolation='none', cmap=cmap,  norm=norm)
    ax[n1].set_title(labels[n1])
    ax[n1].tick_params(axis="x", bottom=True, top=False, labelbottom=True, 
        labeltop=False)  
    ax[n1].set_xlabel('m')
ax[0].set_ylabel('n', rotation=0, labelpad=10)
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size=f"{colourbar_size_percentage}%", pad=colourbar_pad)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)# label="unweighted graph distance")
plt.show()